# Quadrotor (aka "the drone")

## Example of how to set everything up (do this only once)

Import modules.

In [35]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg
import sympy as sym
from sympy import *
import secrets
import ae353_drone

# Suppress the use of scientific notation when printing small numbers
np.set_printoptions(suppress=True)

Create and print a seed for the random number generator so it is possible to reproduce the results. Here are a few best practices:

* By default, keep this code unchanged.
* When producing final results for your report, (1) run this cell once to create a seed, (2) copy/paste this seed into the line of code that creates a simulator, and (3) replace the code cell that creates a seed with a markdown cell saying that you took this approach to choose a seed for testing.

In [2]:
seed = secrets.randbits(32)
print(seed)

1792842405


Create simulator with seed.

If your simulator runs much slower than real-time, then you can try adding the optional argument `display_pybullet=True` (instead of using meshcat views).

In [3]:
simulator = ae353_drone.Simulator(seed=seed)

Add a camera view. This view will be displayed in its own browser window.

In [4]:
simulator.add_view(
    'my_start_view',  # name of view (must be unique)
    'start',          # type of view (start, top, right, left, or back)
)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


## Dynamic model
Define physical parameters.

In [5]:
params = {
    'm': 0.5,
    'Jx': 0.0023,
    'Jy': 0.0023,
    'Jz': 0.0040,
    'l': 0.175,
    'g': 9.81,
}

Derive the equations of motion:

In [6]:
# components of position (meters)
p_x, p_y, p_z = sym.symbols('p_x, p_y, p_z')

# yaw, pitch, roll angles (radians)
psi, theta, phi = sym.symbols('psi, theta, phi')

# components of linear velocity (meters / second)
v_x, v_y, v_z = sym.symbols('v_x, v_y, v_z')
v_in_body = sym.Matrix([v_x, v_y, v_z])

# components of angular velocity (radians / second)
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')
w_in_body = sym.Matrix([w_x, w_y, w_z])

# components of net rotor torque
tau_x, tau_y, tau_z = sym.symbols('tau_x, tau_y, tau_z')

# net rotor force
f_z = sym.symbols('f_z')

# parameters
m = sym.nsimplify(params['m'])
Jx = sym.nsimplify(params['Jx'])
Jy = sym.nsimplify(params['Jy'])
Jz = sym.nsimplify(params['Jz'])
l = sym.nsimplify(params['l'])
g = sym.nsimplify(params['g'])
J = sym.diag(Jx, Jy, Jz)

# rotation matrices
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0], [sym.sin(psi), sym.cos(psi), 0], [0, 0, 1]])
Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)], [0, 1, 0], [-sym.sin(theta), 0, sym.cos(theta)]])
Rx = sym.Matrix([[1, 0, 0], [0, sym.cos(phi), -sym.sin(phi)], [0, sym.sin(phi), sym.cos(phi)]])
R_body_in_world = Rz @ Ry @ Rx

# angular velocity to angular rates
ex = sym.Matrix([[1], [0], [0]])
ey = sym.Matrix([[0], [1], [0]])
ez = sym.Matrix([[0], [0], [1]])
M = sym.simplify(sym.Matrix.hstack((Ry @ Rx).T @ ez, Rx.T @ ey, ex).inv(), full=True)

# applied forces
f_in_body = R_body_in_world.T @ sym.Matrix([[0], [0], [-m * g]]) + sym.Matrix([[0], [0], [f_z]])

# applied torques
tau_in_body = sym.Matrix([[tau_x], [tau_y], [tau_z]])

# equations of motion
f = sym.Matrix.vstack(
    R_body_in_world @ v_in_body,
    M @ w_in_body,
    (1 / m) * (f_in_body - w_in_body.cross(m * v_in_body)),
    J.inv() @ (tau_in_body - w_in_body.cross(J @ w_in_body)),
)

f = sym.simplify(f, full=True)

In [7]:
anum = lambdify((p_x, p_y, p_z, psi, theta, phi,v_x, v_y, v_z,w_x, w_y, w_z,tau_x, tau_y, tau_z, f_z), f.jacobian([p_x, p_y, p_z, psi, theta, phi,v_x, v_y, v_z,w_x, w_y, w_z]))
A = anum(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)
A.shape


array([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,
         0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , -0.  ,  0.  ,  1.  ,
         0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  1.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  1.  , -0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         1.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  9.81,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  , -9.81,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  , -0.  

In [12]:
bnum = lambdify((p_x, p_y, p_z, psi, theta, phi,v_x, v_y, v_z,w_x, w_y, w_z,tau_x, tau_y, tau_z, f_z), f.jacobian([tau_x, tau_y, tau_z, f_z]))
B = bnum(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)
B.shape

(12, 4)

## Sensor model
Define the sensor model.

In [13]:
# Position of drone in world frame
p_in_world = sym.Matrix([p_x, p_y, p_z])

# Position of markers in body frame
a_in_body = sym.Matrix([0, l, 0])  # <-- marker on left rotor
b_in_body = sym.Matrix([0, -l, 0]) # <-- marker on right rotor

# Position of markers in world frame
a_in_world = p_in_world + R_body_in_world @ a_in_body
b_in_world = p_in_world + R_body_in_world @ b_in_body

# Sensor model
g = sym.simplify(sym.Matrix.vstack(a_in_world, b_in_world))
g

Matrix([
[p_x + 7*sin(phi)*sin(theta)*cos(psi)/40 - 7*sin(psi)*cos(phi)/40],
[p_y + 7*sin(phi)*sin(psi)*sin(theta)/40 + 7*cos(phi)*cos(psi)/40],
[                                  p_z + 7*sin(phi)*cos(theta)/40],
[p_x - 7*sin(phi)*sin(theta)*cos(psi)/40 + 7*sin(psi)*cos(phi)/40],
[p_y - 7*sin(phi)*sin(psi)*sin(theta)/40 - 7*cos(phi)*cos(psi)/40],
[                                  p_z - 7*sin(phi)*cos(theta)/40]])

The sensor model has this form:

$$o = g(p_x, p_y, p_z, \psi, \theta, \phi)$$

Here is the function $g$:

In [43]:
cnum = lambdify((p_x, p_y, p_z, psi, theta, phi,v_x, v_y, v_z,w_x, w_y, w_z,tau_x, tau_y, tau_z, f_z),g.jacobian((p_x,p_y,p_z,psi,theta,phi,v_x, v_y, v_z,w_x, w_y, w_z)))
C = cnum(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)
C.shape

(6, 12)

In [23]:
Wc = B
for i in range(1,A.shape[0]):
    col = np.linalg.matrix_power(A,i) @ B
    Wc = np.block([Wc,col])
np.linalg.matrix_rank(Wc) == A.shape[0]

np.True_

In [32]:
Wo = C.T
for i in range(1,A.shape[0]):
    col = np.linalg.matrix_power(A.T,i) @ C.T
    Wo = np.block([Wo,col])
np.linalg.matrix_rank(Wo) == A.shape[0]

np.True_

In [33]:
def lqr(A, B, Q, R):
    # Find the cost matrix
    P = linalg.solve_continuous_are(A, B, Q, R)
    # Find the gain matrix
    K = linalg.inv(R) @ B.T @ P
    return K

In [46]:
Qc = np.eye(12) 
Rc = np.eye(4)

K = lqr(A,B,Qc,Rc)


array([[ 0.        , -1.        ,  0.        ,  0.        ,  0.        ,
         5.46213218, -0.        , -1.45381723,  0.        ,  1.01248497,
         0.        ,  0.        ],
       [ 1.        ,  0.        ,  0.        ,  0.        ,  5.46213218,
         0.        ,  1.45381723,  0.        ,  0.        ,  0.        ,
         1.01248497,  0.        ],
       [ 0.        , -0.        , -0.        ,  1.        ,  0.        ,
         0.        , -0.        , -0.        , -0.        ,  0.        ,
         0.        ,  1.00399203],
       [ 0.        ,  0.        ,  1.        , -0.        ,  0.        ,
        -0.        ,  0.        ,  0.        ,  1.41421356,  0.        ,
         0.        , -0.        ]])

In [45]:
Qo = np.eye(6)
Ro = np.eye(12)

L = lqr(A.T,C.T,linalg.inv(Ro),linalg.inv(Qo))


array([[ 2.97537602,  0.        ,  0.        , -2.13087076,  1.30543174,
        -0.        ,  8.35286245,  0.        , -0.        , -0.        ,
         0.70710678, -0.70710678],
       [ 0.        ,  2.92711379,  0.        , -0.        ,  0.        ,
        -1.25581808,  0.        ,  8.1162932 ,  0.        , -0.67683134,
         0.        , -0.        ],
       [-0.        , -0.21976816,  1.09868411, -0.        , -0.        ,
         0.30429738, -0.        , -1.19153357,  0.70710678,  0.20469329,
        -0.        ,  0.        ],
       [ 2.97537602,  0.        , -0.        ,  2.13087076,  1.30543174,
        -0.        ,  8.35286245,  0.        ,  0.        , -0.        ,
         0.70710678,  0.70710678],
       [ 0.        ,  2.92711379,  0.        , -0.        ,  0.        ,
        -1.25581808,  0.        ,  8.1162932 ,  0.        , -0.67683134,
         0.        , -0.        ],
       [ 0.        ,  0.21976816,  1.09868411,  0.        ,  0.        ,
        -0.30429738,  

## Example of how to add a drone and run a simulation

Clear all drones (there aren't any yet, if you are running this notebook from the start, but we call this function just in case).

In [70]:
simulator.clear_drones()

Define a controller for the drone.

In [71]:
class Controller:
    def __init__(self):
        """
        List all class variables you want the simulator to log. For
        example, if you want the simulator to log "self.xhat", then
        do this:
        
            self.variables_to_log = ['xhat']
        
        Similarly, if you want the simulator to log "self.xhat" and
        "self.y", then do this:
        
            self.variables_to_log = ['xhat', 'y']
        
        Etc. These variables have to exist in order to be logged.
        """
        self.A = A
        self.B = B
        self.C = C
        self.K = K
        self.L = L
        #self.xhat = np.array([0,0,0,0,0,0,0,0,0,0,0,0])
        self.variables_to_log = []

    def get_color(self):
        """
        If desired, change these three numbers - RGB values between
        0 and 1 - to change the color of your drone.
        """
        return [
            0., # <-- how much red (between 0 and 1)
            1., # <-- how much green (between 0 and 1)
            0., # <-- how much blue (between 0 and 1)
        ]

    def reset(
            self,
            p_x, p_y, p_z, # <-- approximate initial position of drone (meters)
            yaw,           # <-- approximate initial yaw angle of drone (radians)
        ):
        """
        Replace the following line (a placeholder) with your
        own code.
        """
        self.xhat = np.array([p_x,p_y,p_z,0,0,yaw,0,0,0,0,0,0])
        pass

    def run(
            self,
            pos_markers,
            pos_ring,
            dir_ring,
            is_last_ring,
            pos_others,
        ):
        """
        pos_markers is a 1d array of length 6:
        
            [
                measured x position of marker on left rotor (meters),
                measured y position of marker on left rotor (meters),
                measured z position of marker on left rotor (meters),
                measured x position of marker on right rotor (meters),
                measured y position of marker on right rotor (meters),
                measured z position of marker on right rotor (meters),
            ]
        
        pos_ring is a 1d array of length 3:
        
            [
                x position of next ring center (meters),
                y position of next ring center (meters),
                z position of next ring center (meters),
            ]
        
        dir_ring is a 1d array of length 3:
        
            [
                x component of vector normal to next ring (meters),
                y component of vector normal to next ring (meters),
                z component of vector normal to next ring (meters),
            ]
        
        is_last_ring is a boolean that is True if the next ring is the
                     last ring, and False otherwise
        
        pos_others is a 2d array of size n x 3, where n is the number of
                   all other active drones:
            
            [
                [x_1, y_1, z_1], # <-- position of 1st drone (meters)
                [x_2, y_2, z_2], # <-- position of 2nd drone (meters)
                
                ...
                
                [x_n, y_n, z_n], # <-- position of nth drone (meters)
            ]      
        """
        pm = np.block([pos_markers,0,0,0,0,0,0])
        y = (self.xhat - pm) # measures - equilibrium points
        u = -self.K@self.xhat

        self.xhat += self.A @ self.xhat + self.B @ u - self.L @ (self.C @ self.xhat - y)
        tau_x = u[0]
        tau_y = u[1]
        tau_z = u[2]
        f_z = u[3]

        return tau_x, tau_y, tau_z, f_z

Add the drone to the simulator. There are three arguments:

* `Controller` is the class definition
* `'template'` is the name of the drone
* `'template.png'` is the name of an image to use as a label (can be `None`, can be a file path)

You can call this function more than once, so long as you call it with a different drone name each time. An exception will be raised if you try to add two drones of the same name to the simulator.

In [72]:
simulator.add_drone(Controller, 'template', 'template.png')

Reset the drone (i.e., place it at a random location in the start ring).

In [73]:
simulator.reset()

Run simulation until `max_time` is reached or until the drone has either reached the finish ring or been disqualified.

In [74]:
simulator.run(
    max_time=5.,       # <-- if None, then simulation will run until all drones fail or finish
    print_debug=True,  # <-- if False, then nothing will be printed (good for data collection)
)


error on run of drone template (turning it off):
Traceback (most recent call last):
  File "C:\Users\lafic\OneDrive\Desktop\AE 353\ae353-sp25\projects\04_drone\ae353_drone.py", line 974, in step
    ) = drone['controller'].run(
        ~~~~~~~~~~~~~~~~~~~~~~~^
        pos_markers,
        ^^^^^^^^^^^^
    ...<3 lines>...
        np.delete(all_pos, np.union1d(index_inactive, index).astype(int), axis=0),
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "C:\Users\lafic\AppData\Local\Temp\ipykernel_2456\1281869515.py", line 103, in run
    self.xhat += self.A @ self.xhat + self.B @ u - self.L @ (self.C @ self.xhat - y)
                                                             ~~~~~~~~~~~~~~~~~~~^~~
ValueError: operands could not be broadcast together with shapes (6,) (12,) 

Simulated 2 time steps in 0.0833 seconds (24.0118 time steps per second)


## Example of how to get and plot results

Assume that we already did two things:

* Added a drone with the name `template` to the simulator
* Ran the simulation

Get result.

In [ ]:
(
    did_it_fail,
    did_it_finish,
    what_time_did_it_finish,
) = simulator.get_result('template')

Show result.

In [ ]:
if did_it_fail:
    print('The drone failed before finishing.')
elif did_it_finish:
    print(f'The drone finished at time {what_time_did_it_finish}')
else:
    print('The drone did not finish (yet).')

Get data.

In [ ]:
data = simulator.get_data('template')

Plot data.

In [ ]:
# Create a figure with subplots that all share the same x-axis
fig, (ax_meas, ax_pos, ax_ori, ax_act) = plt.subplots(4, 1, figsize=(12, 12), sharex=True)

# Measurements
ax_meas.plot(data['t'], data['pos_markers'][:, 0], '.', markersize=4, label=r'$p_{L, x}$')
ax_meas.plot(data['t'], data['pos_markers'][:, 1], '.', markersize=4, label=r'$p_{L, y}$')
ax_meas.plot(data['t'], data['pos_markers'][:, 2], '.', markersize=4, label=r'$p_{L, z}$')
ax_meas.plot(data['t'], data['pos_markers'][:, 3], '.', markersize=4, label=r'$p_{R, x}$')
ax_meas.plot(data['t'], data['pos_markers'][:, 4], '.', markersize=4, label=r'$p_{R, y}$')
ax_meas.plot(data['t'], data['pos_markers'][:, 5], '.', markersize=4, label=r'$p_{R, z}$')
ax_meas.grid()
ax_meas.legend(fontsize=16, bbox_to_anchor=(1., 1.))
ax_meas.tick_params(labelsize=14)

# Position
ax_pos.plot(data['t'], data['p_x'], label='x (m)', linewidth=3, color='C0')
ax_pos.plot(data['t'], data['p_y'], label='y (m)', linewidth=3, color='C1')
ax_pos.plot(data['t'], data['p_z'], label='z (m)', linewidth=3, color='C2')
ax_pos.grid()
ax_pos.legend(fontsize=16, bbox_to_anchor=(1., 1.))
ax_pos.tick_params(labelsize=14)

# Orientation
ax_ori.plot(data['t'], data['yaw'], label='yaw (rad)', linewidth=3)
ax_ori.plot(data['t'], data['pitch'], label='pitch (rad)', linewidth=3)
ax_ori.plot(data['t'], data['roll'], label='roll (rad)', linewidth=3)
ax_ori.grid()
ax_ori.legend(fontsize=16, bbox_to_anchor=(1., 1.))
ax_ori.tick_params(labelsize=14)

# Actuator commands
ax_act.plot(data['t'], data['tau_x'], label='tau_x (N-m)', linewidth=4)
ax_act.plot(data['t'], data['tau_x_cmd'], '--', label='commanded tau_x (N-m)', linewidth=4)
ax_act.plot(data['t'], data['tau_y'], label='tau_y (N-m)', linewidth=4)
ax_act.plot(data['t'], data['tau_y_cmd'], '--', label='commanded tau_y (N-m)', linewidth=4)
ax_act.plot(data['t'], data['tau_z'], label='tau_z (N-m)', linewidth=4)
ax_act.plot(data['t'], data['tau_z_cmd'], '--', label='commanded tau_z (N-m)', linewidth=4)
ax_act.plot(data['t'], data['f_z'] - (0.5 * 9.81), label='f_z - m g (N)', linewidth=4) # <-- FIXME
ax_act.plot(data['t'], data['f_z_cmd'] - (0.5 * 9.81), '--', label='commanded f_z - m g (N)', linewidth=4) # <-- FIXME
ax_act.grid()
ax_act.legend(fontsize=16, bbox_to_anchor=(1., 1.))
ax_act.tick_params(labelsize=14)

# Set shared x-axis properties
ax_act.set_xlabel('time (s)', fontsize=20)
ax_act.set_xlim([data['t'][0], data['t'][-1]])

# Make the arrangement of subplots look nice
fig.tight_layout()
plt.show()

Save the figure (without transparent background). Here are some best practices:

* Use the extension `.png` for rasterized, low-res images (e.g., to include in a notebook or on a website).
* Use the extension `.pdf` for vectorized, high-res images (e.g., to include in a report).

In [ ]:
fig.savefig('my_figure.png', facecolor='white', transparent=False)

Plot histogram of controller run times.

In [ ]:
plt.hist(data['run_time'])
plt.ticklabel_format(style='scientific', scilimits=(0, 0), axis='x')
plt.tick_params(labelsize=14)
plt.xlabel('run time (s)', fontsize=14)
plt.ylabel('count', fontsize=14)
plt.tight_layout()
plt.show()

Note that it is required to also plot state estimates (in comparison to states) in order to check that your observer is working - no example is provided here, since no state estimates are being generated by the template `Controller` class.

## Example of how to load drones from the "students" directory

Clear all drones.

In [ ]:
simulator.clear_drones()

Load drones from `students` directory.

In [ ]:
failures = simulator.load_drones()

Reset drones (places drones at random positions within the start ring).

In [ ]:
simulator.reset()

Run simulation for `max_time` seconds or until all drones finish.

In [ ]:
simulator.run(max_time=5.0)

## Example of how to show results and keep running

Show results (so far).

In [ ]:
simulator.show_results()

Continue running simulation from same point for `max_time` seconds or until all drones either finish or are disqualified.

In [ ]:
simulator.run(max_time=10.0)

Show results (so far).

In [ ]:
simulator.show_results()

## Example of how to clear and reload drones

Clear all drones.

In [ ]:
simulator.clear_drones()

Load drones again from `students` directory.

In [ ]:
failures = simulator.load_drones()

Reset drones (places drones at random positions within the start ring).

In [ ]:
simulator.reset()

Run simulation until `max_time` is reached or until all drones finish.

In [ ]:
simulator.run(max_time=5.0)

## Example of how to rearrange the rings

Rearrange the rings.

In [ ]:
simulator.place_rings()

Reset drones (places drones at random positions within the start ring).

In [ ]:
simulator.reset()

Run simulation until `max_time` is reached or until all drones finish.

In [ ]:
simulator.run(max_time=5.0)

## Example of how to add and remove views

Add another static view in a new browser window.

In [ ]:
simulator.add_view(
    'my_top_view',  # name of view (must be unique)
    'top',          # type of view (start, top, right, left, or back)
)

Add a view of a ring in a new browser window.

In [ ]:
simulator.add_ring_view(
    'my_ring_view', # name of view (must be unique)
    1,              # index of ring (between 1 and 14)
    yaw=180.,       # yaw angle in degrees (0 looks forward, 180 looks backward, etc.)
    distance=2.5,   # how far away the camera is from the center of the ring
)

Add a view of a drone in a new browser window.

In [ ]:
simulator.add_drone_view(
    'my_drone_view',    # name of view (must be unique)
    'template',         # name of drone
    yaw=0.,             # yaw angle in degrees (0 looks forward, 180 looks backward, etc.)
    pitch=15.,          # pitch angle in degrees,
    distance=2.,        # how far away the camera is from the center of the drone
)

Reset drones (places drones at random positions within the start ring).

In [ ]:
simulator.reset()

Run simulation until `max_time` is reached or until all drones finish.

In [ ]:
simulator.run(max_time=5.0)

Remove the top view. The browser window that contains this view will no longer be updated.

In [ ]:
simulator.remove_view(
    'my_top_view',      # name of view to remove
)

Reset drones (places drones at random positions within the start ring). Notice that the top view isn't updated, because it no longer exists.

In [ ]:
simulator.reset()

Add another static view — from the right this time instead of from the top. This new view should appear in the same browser window that formerly contained the top view.

In [ ]:
simulator.add_view(
    'my_right_view',    # name of view (must be unique)
    'right',            # type of view (start, top, right, left, or back)
)

Reset drones (places drones at random positions within the start ring). Notice that the (new) right view *is* updated,

In [ ]:
simulator.reset()

## Example of how to take a snapshot

Display a snapshot.

In [ ]:
# Set the width and height of the snapshot (must be multiples of 16)
simulator.set_snapshot_size(
    640, # <-- width
    480, # <-- height
)

# Get snapshot as height x width x 4 numpy array of RGBA values
rgba = simulator.snapshot(
    'my_start_view',    # name of view from which to take a snapshot
)

# Display snapshot
plt.figure(figsize=(8, 8))
plt.imshow(rgba)

Save the snapshot.

In [ ]:
plt.imsave('my_snapshot.png', rgba)

## Example of how to record a movie

Reset drones (places drones at random positions within the start ring).

In [ ]:
simulator.reset()

Run simulation until `max_time` is reached or until all drones finish, saving movies from two different views. You can add or remove elements to the list of `videos` to save more or fewer movies.

In [ ]:
simulator.run(
    max_time=1.0,
    videos=[
        {
            'view_name': 'my_start_view',       # name of view from which to record a video
            'file_name': 'my_start_video.mp4',  # name of file to which video will be saved
        },
        {
            'view_name': 'my_drone_view',       # name of view from which to record a video
            'file_name': 'my_drone_video.mp4',  # name of file to which video will be saved
        }
    ],
    print_debug=True,   # this option is recommended when saving videos to track progress
)

## Example of how to allow print statements, long run times, and inactivity

By default, your controller will fail if any of the following conditions is met:

* It prints something to `stdout`.
* It exceeds the maximum runtime of `1` second for `init`, `1` second for `reset`, and `1e-2` seconds for `run` (on more than 10 occasions).
* It moves less than `0.1` meters in `10.0` seconds.

You can relax these rules for the purpose of development if you want, as follows.

In [ ]:
simulator.set_rules(
    error_on_print=False,    # <-- allow print statements in controller code
    error_on_timeout=False,  # <-- allow long runtimes in controller code
    error_on_inactive=False, # <-- allow inactivity
)

## Example of how to find the position, orientation, and radius of each ring

This information should only be used for analysis.

In [ ]:
for i, ring in enumerate(simulator.rings):
    p = ring['p']        # <--- position of ring
    v = ring['R'][:, 0]  # <--- unit vector normal to ring (pointing through it)
    r = ring['radius']   # <--- radius of ring
    s = f'ring {i:2d}' + \
        f' : p ({p[0]:6.2f}, {p[1]:6.2f}, {p[2]:6.2f})' + \
        f' : v ({v[0]:6.2f}, {v[1]:6.2f}, {v[2]:6.2f})' + \
        f' : r ({r:6.2f})'
    print(s)

## Example of how to specify initial conditions

Reset the simulator with given initial conditions. The argument `initial_conditions` must be a dictionary. This dictionary must have an key/value pair for each drone in the simulator. The key must be the name of the corresponding drone. The value must, itself, be a dictionary with a key/value pair for each initial state and each initial measurement.

In [ ]:
simulator.reset(
    initial_conditions={
        'template': {
            'p_x': 0.,
            'p_y': 0.,
            'p_z': 0.5,
            'yaw': 0.,
            'pitch': 0.,
            'roll': 0.,
            'v_x': 0.,
            'v_y': 0.,
            'v_z': 0.,
            'w_x': 0.,
            'w_y': 0.,
            'w_z': 0.,
            'p_x_meas': 0.,
            'p_y_meas': 0.,
            'p_z_meas': 0.5,
            'yaw_meas': 0.,
        },
    },
)

Run simulation for `max_time` seconds or until all drones finish.

In [ ]:
simulator.run(max_time=5.0)

## Example of how to disable and enable views

Disable views so they aren't updated while running simulations. These simulations will then run as fast as possible (much faster than real time), so this option is a good choice when collecting lots of data.

In [ ]:
simulator.disable_views()

Reset drones (places drones at random positions within the start ring). Note that views are still updated — it is only when running simulations that views aren't updated.

In [ ]:
simulator.reset()

Run simulation for `max_time` seconds or until all drones finish (should take much less than `max_time` seconds to run).

In [ ]:
simulator.run(max_time=5.0)

Enable views again.

In [ ]:
simulator.enable_views()

Run simulation for `max_time` seconds or until all drones finish (should take about `max_time` seconds to run).

In [ ]:
simulator.run(max_time=5.0)